In [1]:
import torch
import torch.nn.functional as F
words = open("names.txt", 'r').read().splitlines()

In [2]:
N = torch.zeros((27,27), dtype=torch.int32)
chars = sorted(list(set(''.join(words))))
c_to_i = {c:i+1 for i,c in enumerate(chars)}
c_to_i['.'] = 0
i_to_c = {i+1:c for i,c in enumerate(chars)}
i_to_c[0] = '.'

In [3]:
for word in words:
    chs = ['.'] + list(word) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        N[c_to_i[c1]][c_to_i[c2]] += 1

In [4]:
print(N[:,0])
print(i_to_c)

tensor([   0, 6640,  114,   97,  516, 3983,   80,  108, 2409, 2489,   71,  363,
        1314,  516, 6763,  855,   33,   28, 1377, 1169,  483,  155,   88,   51,
         164, 2007,  160], dtype=torch.int32)
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
P = (N+1).float()
P /= P.sum(1, True)

In [6]:
#generate 1 name
gen = torch.Generator().manual_seed(25)
for i in range(10):
    x = 0
    chr_list = []
    while (len(chr_list) == 0 or x != 0):
        x = torch.multinomial(P[x], 1, True, generator=gen).item()
        chr_list.append(i_to_c[x])
    chr_list.pop()
    name = ''.join(chr_list)
    print(name)

kauwanax
eijan
danin
sol
lidetoma
tie
al
sala
e
shamele


In [47]:
x_train = []
y_train = []

for word in words:
    chrs = ['.'] + list(word) + ['.']
    for chr1, chr2 in zip(chrs, chrs[1:]):
        x_train.append(c_to_i[chr1])
        y_train.append(c_to_i[chr2])

x_train = F.one_hot(torch.tensor(x_train), 27).float()
y_train = torch.tensor(y_train)

In [98]:
weights = torch.randn((27,27), requires_grad=True)

loss
for i in range(10):
    raw_activations = (x_train @ weights)
    exp_activations = raw_activations.exp()
    prob = exp_activations / exp_activations.sum(1, True)
    
    loss = -prob[torch.arange(len(prob)), y_train].log().mean()
    weights.grad = None
    loss.backward()
    weights.data += -5.0 * weights.grad
    
print(loss.item())
print(prob)

3.403341293334961
tensor([[0.0568, 0.0272, 0.0548,  ..., 0.0074, 0.0551, 0.0399],
        [0.1253, 0.0393, 0.0187,  ..., 0.0231, 0.0108, 0.0215],
        [0.0352, 0.0191, 0.0199,  ..., 0.0193, 0.0518, 0.0997],
        ...,
        [0.0067, 0.0507, 0.0391,  ..., 0.0074, 0.1066, 0.0280],
        [0.0036, 0.0184, 0.1180,  ..., 0.0129, 0.0279, 0.0115],
        [0.0385, 0.0289, 0.0429,  ..., 0.0505, 0.0141, 0.0481]],
       grad_fn=<DivBackward0>)


In [99]:
for i in range(10):
    x = 0
    chr_list = []
    while (len(chr_list) == 0 or x != 0):
        x_val = (F.one_hot(torch.tensor([x]), num_classes = 27).float() @ weights).exp()
        p = x_val / x_val.sum(1, True)
        x = torch.multinomial(p, 1, True, generator=gen).item()
        chr_list.append(i_to_c[x])
    name = ''.join(chr_list)
    print(name)

xfbggggkanfbuzi.
tffgfpnucyvdtoyyzwcgglwfswteahhsxgbyynumzkmxyzfvysgei.
ychtchcpmlwcbmkmobulpwtoqqt.
bkpe.
ihodears.
jqjyalrmhvhftcfyqnhs.
jkpjamvlhqvuezwcqvuzbgykcfovvivqpdatowacpxifk.
bkroyhhcbmlwpkqqsjrwoxginuromuypnulltctwjjditcqsxfxgrepfylfhtqqlrb.
zwpyuzrefitqdgnuychfyml.
zmgrulobawcsvqpn.


In [100]:
prob.shape

torch.Size([228146, 27])